<a href="https://colab.research.google.com/github/bkvkrll/Recommender-systems/blob/main/Lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
pip install implicit==0.4.4

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [47]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [48]:
data['week_no'].nunique()

95

In [49]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [50]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [51]:
item_features = pd.read_csv('product.csv')
item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [52]:
user_features = pd.read_csv('hh_demographic.csv')
user_features.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [53]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [54]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [55]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [56]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


1.1 Random recommendation

In [57]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [58]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 3.43 s, sys: 31.6 ms, total: 3.47 s
Wall time: 3.51 s


1.2 Popularity-based recommendation

In [59]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [60]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 116 ms, sys: 1.9 ms, total: 118 ms
Wall time: 115 ms


1.3 Weighted random recommender

In [61]:
items_weights = data_train.groupby('item_id').agg(sales=('sales_value', 'sum')).reset_index()
items_weights['weights'] = items_weights[['sales']].apply(lambda x: x / np.sum(x))
items_weights

,item_id,sales,weights
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07
2,26093,1.59,2.254623e-07
3,26190,1.54,2.183723e-07
4,26355,1.98,2.807644e-07
...,...,...,...
86860,17381856,0.00,0.000000e+00
86861,17382205,7.99,1.132983e-06
86862,17383227,4.49,6.366828e-07
86863,17827644,2.50,3.545005e-07


In [62]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    items_weights = np.array(items_weights['weights'])
    recs = np.random.choice(items, size=n, replace=False)
    
    
    return recs.tolist()

In [63]:
%%time


result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result['weighted_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))



CPU times: user 4.92 s, sys: 91.7 ms, total: 5.02 s
Wall time: 6.59 s


In [64]:
result.head(2)

,user_id,actual,weighted_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1133727, 913425, 9368282, 1800064, 56918]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1432456, 12673005, 947492, 13382462, 10355637]"


In [65]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [66]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0007:weighted_recommendation


Задание 2. Улучшение бейзлайнов и ItemItem
Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  K .

In [67]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [68]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [69]:
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [70]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [71]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [72]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [73]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [74]:
%%time
for i in range(4,8):
    model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'itemitem_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

CPU times: user 6.76 s, sys: 27.1 ms, total: 6.78 s
Wall time: 5 s


In [75]:
%%time
for i in range(4,8):
    model = CosineRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'cosine_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

CPU times: user 6.89 s, sys: 32.5 ms, total: 6.92 s
Wall time: 4.09 s


In [76]:
%%time
for i in range(4,8):
    model = TFIDFRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'tfidf_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

CPU times: user 6.86 s, sys: 18.1 ms, total: 6.87 s
Wall time: 4.03 s


In [77]:
result.shape,sparse_user_item.shape

((2042, 15), (2499, 5001))

In [78]:
result.head(2)

,user_id,actual,weighted_recommendation,itemitem_k_4,itemitem_k_5,itemitem_k_6,itemitem_k_7,cosine_k_4,cosine_k_5,cosine_k_6,cosine_k_7,tfidf_k_4,tfidf_k_5,tfidf_k_6,tfidf_k_7
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1133727, 913425, 9368282, 1800064, 56918]","[6666, 1082185, 981760, 995242, 1127831]","[6666, 1082185, 981760, 1127831, 995242]","[6666, 1082185, 981760, 1127831, 995242]","[6666, 1082185, 981760, 995242, 840361]","[1082185, 6666, 981760, 1127831, 961554]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 6666, 981760, 1127831, 961554]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 961554]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 981760, 1127831, 6666, 961554]","[1082185, 981760, 1127831, 961554, 6666]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1432456, 12673005, 947492, 13382462, 10355637]","[6666, 1082185, 981760, 1098066, 826249]","[6666, 1082185, 981760, 1098066, 995242]","[6666, 1082185, 981760, 1098066, 826249]","[6666, 1082185, 981760, 1098066, 826249]","[1082185, 1098066, 6666, 981760, 826249]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 6666, 826249]","[1082185, 1098066, 981760, 826249, 6666]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 1098066, 981760, 826249, 6666]","[1082185, 981760, 1098066, 826249, 6666]"
